In [ ]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)
print(tf.test.gpu_device_name())


from keras.datasets import fashion_mnist, cifar10
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

2.8.0
2.8.0
/device:GPU:0


2022-11-18 13:53:42.765985: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 13:53:43.478975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:53:43.504797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:53:43.504944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

## Fashion MNIST

Load and split the data (fashion MNIST).

In [ ]:
# the data, split between train and test sets
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

4431872/4422102 [==============================] - 0s 0us/step


Train and evaluate an MLP.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30


2022-11-18 13:55:27.725074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:55:27.725334: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:55:27.725511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:55:27.725932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 13:55:27.726126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

 105/1719 [>.............................] - ETA: 2s - loss: 1.7412 - accuracy: 0.5065

2022-11-18 13:55:28.868497: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1719/1719 [==============================] - 4s 2ms/step - loss: 0.7337 - accuracy: 0.7620 - val_loss: 0.5075 - val_accuracy: 0.8318
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4858 - accuracy: 0.8316 - val_loss: 0.4436 - val_accuracy: 0.8524
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4391 - accuracy: 0.8461 - val_loss: 0.4352 - val_accuracy: 0.8486
Epoch 4/30
1719/1719 [==============================] - 3s 1ms/step - loss: 0.4110 - accuracy: 0.8566 - val_loss: 0.4041 - val_accuracy: 0.8620
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3920 - accuracy: 0.8627 - val_loss: 0.3880 - val_accuracy: 0.8680
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3767 - accuracy: 0.8673 - val_loss: 0.3769 - val_accuracy: 0.8674
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3620 - accuracy: 0.8718 - val_loss: 0.3708 - val_accuracy: 0.8684
Epo

In [ ]:
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

Train and evaluate a CNN.

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv2D(64, 7, activation="relu", padding="same",
  input_shape=[28, 28, 1]),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
  keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
  keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation="relu"),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(64, activation="relu"),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(10, activation="softmax")
])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)        

2022-11-18 13:57:04.719365: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-11-18 13:57:05.382326: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-18 13:57:05.382980: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-18 13:57:05.382990: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-18 13:57:05.383373: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-18 13:57:05.383404: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


430/430 [==============================] - 7s 10ms/step - loss: 0.9299 - accuracy: 0.6601 - val_loss: 0.3681 - val_accuracy: 0.8642
Epoch 2/12
430/430 [==============================] - 4s 10ms/step - loss: 0.4421 - accuracy: 0.8499 - val_loss: 0.3342 - val_accuracy: 0.8818
Epoch 3/12
430/430 [==============================] - 4s 9ms/step - loss: 0.3557 - accuracy: 0.8787 - val_loss: 0.2794 - val_accuracy: 0.9040
Epoch 4/12
430/430 [==============================] - 4s 9ms/step - loss: 0.3173 - accuracy: 0.8953 - val_loss: 0.2857 - val_accuracy: 0.9036
Epoch 5/12
430/430 [==============================] - 4s 9ms/step - loss: 0.2918 - accuracy: 0.9030 - val_loss: 0.2912 - val_accuracy: 0.8932
Epoch 6/12
430/430 [==============================] - 4s 9ms/step - loss: 0.2747 - accuracy: 0.9098 - val_loss: 0.2663 - val_accuracy: 0.9096
Epoch 7/12
430/430 [==============================] - 4s 9ms/step - loss: 0.2662 - accuracy: 0.9124 - val_loss: 0.3404 - val_accuracy: 0.9086
Epoch 8/12
430/

### CIFAR-10

In [ ]:
# the data, split between train and test sets
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 3072)              0         
                                                                 
 dense_41 (Dense)            (None, 300)               921900    
                                                                 
 dense_42 (Dense)            (None, 100)               30100     
                                                                 
 dense_43 (Dense)            (None, 10)                1010      
                                                                 
Total params: 953,010
Trainable params: 953,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1407/1407 [==============================] - 5s 3ms/step - loss: 1.8844 - accuracy: 0.3224 - val_loss: 2.0321 - val_accuracy: 0.3108
Epoch 2/30
1407/1407 [=============

In [ ]:
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv2D(64, 7, activation="relu", padding="same",
  input_shape=[32, 32, 3]),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
  keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
  keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
  keras.layers.MaxPooling2D(2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(seed=None)),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(64, activation="relu", kernel_initializer=tf.keras.initializers.HeNormal(seed=None)),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(10, activation="softmax", kernel_initializer=tf.keras.initializers.HeNormal(seed=None))
])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 32, 32, 64)        9472      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_42 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 8, 8, 256)       